In [1]:
# (ignore this) imports and utility functions

import re
import json
import toolkit

from IPython.display import clear_output, display, Markdown


def render_markdown(text):
    if text is None:
        text = ""

    pattern = r"```thinking\n(.*?)\n```"

    def replacement(match):
        content = match.group(1)
        return f"<blockquote>\n{content}\n</blockquote>\n\n"

    text = re.sub(pattern, replacement, text, flags=re.DOTALL)

    text = re.sub(r"```function_spec\b", "```json", text)
    text = re.sub(r"```function_call\b", "```json", text)
    text = re.sub(r"```function_output\b", "```json", text)

    display(Markdown(text))


def render_json(data):
    return render_markdown(f"```json\n{json.dumps(data, indent=2)}\n```\n")


def render_live(stream):
    response = ""
    render_markdown("> Loading...")

    for chunk in stream:
        response += chunk
        clear_output(wait=True)
        render_markdown(response)

    return response

In [2]:
model = "gemma3:27b"

In [3]:
await toolkit.code.create_sandbox()

In [4]:
async def run_python_code(code):
    runner = await toolkit.code.sandbox.run(code)
    output = await runner.output()
    errors = await runner.error()

    if errors:
        return {
            "error": {
                "name": "RuntimeException",
                "message": f"Function execution produced runtime errors: {errors}",
            }
        }

    return {"result": output}

In [5]:
spec = {
    "name": "run_python_code",
    "description": "Executes a given Python code string in a secure, sandboxed environment and returns the standard output. This is useful for simple and complex scripts that allow you to perform calculations, data manipulation, or interacting with external APIs via network requests. If you do not have access to a function, write python code for the function and run it by calling this function instead.",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {
                "type": "string",
                "description": "A string containing the valid Python code to be executed.",
            }
        },
        "required": ["code"],
    },
    "responses": [
        {
            "type": "string",
            "description": "The standard output (stdout) generated by the successful execution of the code.",
        }
    ],
    "errors": [
        {
            "name": "Exception",
            "description": "Raised if the executed code produces any runtime errors or exceptions. The error message will contain the details from stderr.",
        }
    ],
}

In [6]:
toolkit.functions.register(run_python_code, spec)

In [7]:
messages = []
prompt = toolkit.prompts.function_calling(model)
specs = toolkit.functions.specify()
task = "What's the weather in Pune? Use the wttr.in/<place> API please."

In [8]:
messages.append({"role": "user", "content": prompt})
messages.append({"role": "user", "content": specs})
messages.append({"role": "user", "content": task})

In [9]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
I need to find the weather in Pune. The user has specifically asked me to use the wttr.in/<place> API. Since I don't have a dedicated function to directly access this API, I need to use the `run_python_code` function to execute Python code that fetches the weather data from the wttr.in API. I will construct the URL, fetch the data, and return the output.
</blockquote>


```json
{
	"id": "1",
	"function": "run_python_code",
	"parameters": {
		"code": "import requests\n\nurl = 'http://wttr.in/Pune'\nresponse = requests.get(url)\n\nif response.status_code == 200:\n    print(response.text)\nelse:\n    print(f\"Error: {response.status_code}\")"
	}
}
```

In [10]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
messages.append({"role": "user", "content": outputs})
render_markdown(outputs)

```json
{
    "id": "1",
    "result": ""
}
```


In [12]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
The `run_python_code` function executed successfully but returned an empty string. This likely means the request to wttr.in didn't produce any output, or there was an error during the execution that wasn't explicitly caught and reported. I should re-run the python code, but this time I'll add explicit error handling to catch any exceptions that might occur during the request or response processing.
</blockquote>


```json
{
	"id": "2",
	"function": "run_python_code",
	"parameters": {
		"code": "import requests\n\ntry:\n    url = 'http://wttr.in/Pune'\n    response = requests.get(url)\n    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)\n    print(response.text)\nexcept requests.exceptions.RequestException as e:\n    print(f\"Error: {e}\")"
	}
}
```

In [13]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
messages.append({"role": "user", "content": outputs})
render_markdown(outputs)

```json
{
    "id": "2",
    "result": "Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))"
}
```


In [14]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
The `run_python_code` function again returned an error. This time, the error message indicates a connection issue: "Connection aborted. RemoteDisconnected('Remote end closed connection without response')". This suggests a network problem or that the wttr.in service is temporarily unavailable. Since I've tried fetching the weather information twice and both attempts failed with connection-related errors, it's likely a temporary issue with the external service or the network connection. I will inform the user that I was unable to retrieve the weather information due to a connection error and suggest they try again later.
</blockquote>



I was unable to retrieve the weather information for Pune. There appears to be a connection error. The wttr.in service might be temporarily unavailable, or there might be a network issue. Please try again later.